<font size = 10>

# pimm依赖

## ***data系列***

<font size = 3>

### pimm.data.Dataset 与 pimm.data.create_loader

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.data import Dataset, create_loader

#### - 测试主要方法与函数运行

In [ ]:
dataset_eval = Dataset("Cream/data/imagenet/val")

In [ ]:
loader_eval = create_loader(
        dataset_eval,
        input_size = (3, 224, 224),
        batch_size = 4 * 32,
        is_training = True,
        num_workers = 4,
        distributed = False,
        interpolation = 'bicubic',
        crop_pct = 0.875,
        mean = (0.485, 0.456, 0.406),
        std = (0.229, 0.224, 0.225))

#### - 输出值测试

In [ ]:
for _, (input, _) in enumerate(loader_eval):
    break
print(input)

## ***loss系列***

<font size = "3">

### pimm.loss.LabelSmoothingCrossEntropy

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.loss import LabelSmoothingCrossEntropy

#### - 测试方法创建

In [ ]:
pd_fun = LabelSmoothingCrossEntropy()

#### - 前向对齐测试

In [ ]:
import numpy as np

In [ ]:
output = np.random.rand(100,20)
target = np.random.randint(low = 0, high = 20, size = (100))
for i in range(100):
    output[i][target[i]] += 10
    output[i] /= output[i].sum()

In [ ]:
import paddle
pd_loss = pd_fun(
    paddle.to_tensor(output, dtype = "float32"), 
    paddle.to_tensor(target))
print(pd_loss)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, smoothing=0.1):
        super(LabelSmoothingCrossEntropy, self).__init__()
        assert smoothing < 1.0
        self.smoothing = smoothing
        self.confidence = 1. - smoothing

    def forward(self, x, target):
        logprobs = F.log_softmax(x, dim=-1)
        nll_loss = -logprobs.gather(dim=-1, index=target.unsqueeze(1))
        nll_loss = nll_loss.squeeze(1)
        smooth_loss = -logprobs.mean(dim=-1)
        loss = self.confidence * nll_loss + self.smoothing * smooth_loss
        return loss.mean()

In [ ]:
index = torch.from_numpy(target).type_as(torch.zeros((2,2), dtype = torch.int64))
tc_loss = LabelSmoothingCrossEntropy()(
    torch.from_numpy(output), 
    index)
print(tc_loss)

## ***models.effcientnet_blocks系列***

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models.efficientnet_blocks import ConvBnAct, DepthwiseSeparableConv, drop_path, InvertedResidual,SqueezeExcite

<font size = 3>

### pimm.models.effcientnet_blocks.ConvBnAct

#### - 网络创建

In [ ]:
conv_bn_act = ConvBnAct(in_chs = 3, out_chs = 8, kernel_size = 3)

#### - 网络组网与运行测试

In [ ]:
import paddle
conv_bn_act_model = paddle.Model(conv_bn_act)
conv_bn_act_model.summary(input_size = (10, 3, 224, 224))

In [ ]:
class ConvBnAct_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(ConvBnAct_Model, self).__init__()
        self.core = ConvBnAct(in_chs = 3, out_chs = 8, kernel_size = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

conv_bn_act_model = ConvBnAct_Model()
conv_bn_act_model.eval()

conv_bn_act_model(paddle.randn((10, 3, 224, 224)))

<font size = 3>

### pimm.models.effcientnet_blocks.DepthwiseSeparableConv

#### - 网络创建

In [ ]:
dsc = DepthwiseSeparableConv(in_chs = 8, out_chs = 3)

#### - 网络组网与运行测试

In [ ]:
import paddle
dsc_model = paddle.Model(dsc)
dsc_model.summary(input_size = (1, 8, 224, 224))

In [ ]:
import paddle
class DSC_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(DSC_Model, self).__init__()
        self.core = DepthwiseSeparableConv(in_chs = 8, out_chs = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

dsc_model = DSC_Model()
dsc_model.eval()

dsc_model(paddle.randn((10, 8, 224, 224)))

<font size = 3>

### pimm.models.effcientnet_blocks.drop_path

In [ ]:
import paddle
test_Tensor = paddle.randn((16, 3, 224, 224))
drop_path(test_Tensor)

<font size = 3>

### pimm.models.effcientnet_blocks.InvertedResidual

#### - 网络创建

In [ ]:
ir = InvertedResidual(in_chs = 8, out_chs = 3)

#### - 网络组网与运行测试

In [ ]:
import paddle
ir_model = paddle.Model(ir)
ir_model.summary(input_size = (16, 8, 224, 224))

In [ ]:
import paddle
class IR_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(IR_Model, self).__init__()
        self.core = InvertedResidual(in_chs = 8, out_chs = 3)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

ir_model = IR_Model()
ir_model.eval()

ir_model(paddle.randn((16, 8, 224, 224)))

<font size = 3>

### pimm.models.effcientnet_blocks.SqueezeExcite

#### - 网络创建

In [ ]:
se = SqueezeExcite(in_chs = 8)

#### - 网络组网与运行测试

In [ ]:
import paddle
se_model = paddle.Model(se)
se_model.summary(input_size = (16, 8, 224, 224))

In [ ]:
import paddle
class SE_Model(paddle.nn.Layer):
    def __init__(self) -> None:
        super(SE_Model, self).__init__()
        self.core = SqueezeExcite(in_chs = 8)
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

se_model = SE_Model()
se_model.eval()

se_model(paddle.randn((16, 8, 224, 224)))

## ***models.activations系列***

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models.activations import hard_sigmoid, Swish

<font size = 3>

### pimm.models.activations.hard_sigmoid

In [ ]:
import paddle
import numpy as np
hard_sigmoid(paddle.to_tensor(np.linspace(0., 10.)))

<font size = 3>

### pimm.models.activations.Swish

In [ ]:
import paddle
import numpy as np
swish = Swish()
swish(paddle.to_tensor(np.linspace(0., 10.)))

## ***models系列（其他函数与类）***

<font size = 3>

### pimm.models.create_conv2d

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import create_conv2d

#### - 调用测试

In [ ]:
Conv_2D = []
Conv_2D.append(create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7]))
Conv_2D.append(create_conv2d(in_chs = 8, out_chs = 3, kernel_size = 5, num_experts = 3))
Conv_2D.append(create_conv2d(in_chs = 8, out_chs = 3, kernel_size = 5))

#### - 网络结构检查

In [ ]:
import paddle
for module in (Conv_2D[0], Conv_2D[2]):
    model_test = paddle.Model(module)
    model_test.summary(input_size = (16, 8, 224, 224))
    del model_test

<font size = 3>

### pimm.models.SelectAdaptivePool2D

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import SelectAdaptivePool2D

#### - 网络创建

In [ ]:
pools = [
    SelectAdaptivePool2D(output_size = 10, pool_type = "avgmax"), 
    SelectAdaptivePool2D(output_size = 10, pool_type = "catavgmax"), 
    SelectAdaptivePool2D(output_size = 10, pool_type = "avg"), 
    SelectAdaptivePool2D(output_size = 10, pool_type = "max")]

#### - 网络结构检查

In [ ]:
import paddle
for layer in pools:
    model = paddle.Model(layer)
    model.summary(input_size = (16, 3, 224, 224))

#### - 网络组网与运行测试

In [ ]:
import paddle
test_x = paddle.randint(0, 2, (1, 2, 3, 3))
test_x = paddle.to_tensor(test_x, dtype = "float32")
print(test_x)

In [ ]:
class Pool_Model_1(paddle.nn.Layer):
    def __init__(self) -> None:
        super(Pool_Model_1, self).__init__()
        self.core = SelectAdaptivePool2D(output_size = 10, pool_type = "avgmax")
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

class Pool_Model_2(paddle.nn.Layer):
    def __init__(self) -> None:
        super(Pool_Model_2, self).__init__()
        self.core = SelectAdaptivePool2D(output_size = 10, pool_type = "catavgmax")
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

class Pool_Model_3(paddle.nn.Layer):
    def __init__(self) -> None:
        super(Pool_Model_3, self).__init__()
        self.core = SelectAdaptivePool2D(output_size = 10, pool_type = "avg")
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

class Pool_Model_4(paddle.nn.Layer):
    def __init__(self) -> None:
        super(Pool_Model_4, self).__init__()
        self.core = SelectAdaptivePool2D(output_size = 10, pool_type = "max")
    
    @paddle.jit.to_static
    def forward(self, x):
        return self.core(x)

Pool_Models = [Pool_Model_1, Pool_Model_2, Pool_Model_3, Pool_Model_4]

In [ ]:
for test_Model in Pool_Models:
    model = test_Model()
    model.eval()
    print(model(test_x))

<font size = 3>

### pimm.models.resume_checkpoint

测试此部分前，请先测试`pimm.utils.CheckpointSaver`以生成本部分测试所需要的存档

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import resume_checkpoint

#### - 存档载入测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import create_conv2d
test_Model = create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7])

resume_checkpoint(test_Model, "test/last.pdparams")

测试完成后请运行`utils.CheckpointSaver`中的“删除生成文件部分以”删除用于测试的存档

## ***utils系列***

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.utils import reduce_tensor, CheckpointSaver, ModelEma

<font size = 3>

### pimm.utils.reduce_tensor

In [ ]:
import paddle
test_Tensor = paddle.randn((3, 3, 3))
print(test_Tensor)

In [ ]:
reduced_test_Tensor = reduce_tensor(test_Tensor, 3.)
print(reduced_test_Tensor)

<font size = 3>

### pimm.utils.CheckpointSaver

#### - 实例化测试

In [ ]:
saver = CheckpointSaver(checkpoint_dir = "test", recovery_dir = "test", max_history = 5)

#### - 存储测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import create_conv2d
import paddle

test_Model = create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7])
test_Optimizer = paddle.optimizer.Adam(parameters = test_Model.parameters())

class test_Args:
    def __init__(self):
        self.model = ""

test_args = test_Args()

In [ ]:
import random
for i in range(20):
    saver.save_checkpoint(
    model = test_Model, optimizer = test_Optimizer, 
    args = test_args, epoch = i + 1, 
    metric = random.random())

In [ ]:
for checkpoint in saver.checkpoint_files:
    print(checkpoint)

#### - 删除生成文件（若要测试resume_checkpoint则不运行）

In [ ]:
import os
f_dir = os.getcwd()
l_dir = os.path.join(f_dir, "test")
if os.path.exists(l_dir):
    lis_dir = os.listdir(l_dir)
    for tar_dir in lis_dir:
        os.remove(os.path.join(l_dir, tar_dir))
    os.removedirs(l_dir)

<font size = 3>

### pimm.utils.ModelEma

#### - 实例化测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import create_conv2d
ema = ModelEma(
    create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7]))

#### - 更新测试

In [ ]:
ema.update(
    create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7]))

## ***optim与scheduler系列***

<font size = 3>

### pimm.optim.create_optimizer

没有有效的测试方法

<font size = 3>

### pimm.scheduler.create_scheduler

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pimm.scheduler import create_scheduler

#### - 实例化测试

In [ ]:
class test_Args:
    def __init__(self):
        self.sched = "step"
        self.lr = .01
        self.epochs = 200
        self.decay_epochs = 4
        self.decay_rate = .1
        self.warmup_lr = 1e-4
        self.warmup_epochs = 3
        self.noise_range = None
        self.lr_noise_pct = .67
        self.lr_noise_std = 1.
        self.seed = 42

In [ ]:
sched, num_epochs = create_scheduler(test_Args())

In [ ]:
for i in range(20):
    sched.step(i)
    print(sched.get_lr())

# phop依赖

<font size = 3>

### phop.profile

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.phop import profile

#### - 调用测试

In [ ]:
import paddle
from Paddle_Cream.lib.utils.pimm.models import create_conv2d
test_layer = create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7])
test_model = paddle.Model(test_layer)
test_model.summary(input_size = (16, 8, 224, 224))

In [ ]:
test_input = paddle.randn((16, 8, 224, 224))
print(
    profile(
        test_layer, 
        inputs = test_input))

In [ ]:
test_layer.buffers()

# pdflops依赖

<font size = 3>

### pdflops.get_model_complexity_info

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.pdflops import get_model_complexity_info

#### - 调用测试

In [ ]:
import paddle
from Paddle_Cream.lib.utils.pimm.models import create_conv2d
test_layer2 = create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7])
test_layer1 = paddle.nn.Linear(in_features = 20, out_features = 8)

In [ ]:
re1 = get_model_complexity_info(
    test_layer1, 
    (1, 20), 
    as_strings = False, 
    print_per_layer_stat = False)
re2 = get_model_complexity_info(
    test_layer2, 
    (8, 224, 224), 
    as_strings = False, 
    print_per_layer_stat = False)

In [ ]:
print(re1)
print(re2)

#### - 结果核对

In [ ]:
import torch
from timm.models.layers import create_conv2d
test_layer1_t = torch.nn.Linear(in_features = 20, out_features = 8)
test_layer2_t = create_conv2d(in_chs = 8, out_chs = 3, kernel_size = [3, 5, 7])

In [ ]:
from Cream.lib.utils.flops_counter import get_model_complexity_info
re1 = get_model_complexity_info(
    test_layer1_t, 
    (1, 20), 
    as_strings = False, 
    print_per_layer_stat = False)
re2 = get_model_complexity_info(
    test_layer2_t, 
    (8, 224, 224), 
    as_strings = False, 
    print_per_layer_stat = False)

In [ ]:
print(re1)
print(re2)

# [lib/models](Paddle_Cream\lib\models)

## ***[blocks](Paddle_Cream\lib\models\blocks)***

#### - import测试

In [ ]:
from Paddle_Cream.lib.models.blocks import InvertedResidual, get_BasicBlock, get_Bottleneck

<font size = 4>

### InvertedResidual

#### - 实例化测试

In [ ]:
invert_resi_block = InvertedResidual(
    in_chs = 3, out_chs = 16, 
    dw_kernel_size = 3, stride = 1, dilation = 1,
    exp_kernel_size = 1, pw_kernel_size=1, se_ratio=0., 
    drop_path_rate=0.)

#### - 结构检查

In [ ]:
import paddle
invert_resi_model = paddle.Model(invert_resi_block)
invert_resi_model.summary(input_size = (16, 3, 224, 224))

#### - 网络组网与运行测试

In [ ]:
invert_resi_block.eval()
test_input = paddle.randn((16, 3, 224, 224))
invert_resi_block(test_input)

<font size = 4>

### get_BasicBlock()

#### - 调用测试

In [ ]:
basic_block = get_BasicBlock(3, 3, 1)

#### - 网络结构检查

In [ ]:
import paddle
basic_block_model = paddle.Model(basic_block)
basic_block_model.summary((16, 3, 224, 224))

#### - 网络运行测试

In [ ]:
basic_block.eval()
test_input = paddle.randn((16, 3, 224, 224))
basic_block(test_input)

<font size = 4>

### get_Bottleneck

#### - 调用测试

In [ ]:
bottleneck_block = get_Bottleneck(3, 8, 2)

#### - 网络结构检查

In [ ]:
import paddle
bottleneck_model = paddle.Model(bottleneck_block)
bottleneck_model.summary((16, 3, 224, 224))

#### - 网络运行测试

In [ ]:
bottleneck_block.eval()
test_input = paddle.randn((16, 3, 224, 224))
bottleneck_block(test_input)

## ***[builders](Paddle_Cream\lib\models\builders)***

#### - import测试

In [ ]:
from Paddle_Cream.lib.models.builders.build_childnet import ChildNetBuilder
from Paddle_Cream.lib.models.builders.build_supernet import SuperNetBuilder

<font size = 3>

### ChildNetBuilder

#### - 实例化测试

In [ ]:
childnet_block = ChildNetBuilder()

#### - 调用测试

In [ ]:
import paddle
test_args = [[{
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ds",  
        "dw_kernel_size": 3, 
        "out_chs": 16, 
        "stride": 2
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "exp_ratio": 6, 
        "out_chs": 24, 
        "stride": 2
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "out_chs": 40, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "exp_ratio": 6, 
        "out_chs": 40, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "exp_ratio": 6, 
        "out_chs": 80, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "exp_ratio": 6, 
        "out_chs": 80, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 3, 
        "exp_ratio": 6, 
        "out_chs": 96, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "ir", 
        "dw_kernel_size": 5, 
        "exp_ratio": 6, 
        "out_chs": 192, 
        "stride": 1
    }, {
        "act_layer": paddle.nn.ReLU, 
        "block_type": "cn", 
        "kernel_size": 1, 
        "out_chs": 40, 
        "stride": 1
    }
]]

In [ ]:
test_stages = childnet_block(16, test_args)

#### - 组网测试

In [ ]:
test_models = []
for stage in test_stages:
    test_models.append(paddle.Model(stage))

In [ ]:
test_models[0].summary((16, 16, 32, 32))

<font size = 3>

### SuperNetBuilder

#### - 实例化测试

In [ ]:
supernet_block = SuperNetBuilder({'kernel_size': [3, 5, 7], 'exp_ratio': [4, 6]})

#### - 调用测试

In [ ]:
test_stages = supernet_block(16, test_args)

In [ ]:
test_stages

## ***[structures](Paddle_Cream\lib\models\structures)***

#### - import测试

In [1]:
from Paddle_Cream.lib.models.structures.supernet import gen_supernet
from Paddle_Cream.lib.models.structures.childnet import gen_childnet

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\torchvision\transforms\functional.py:5: DeprecationWarning: PILLOW_VERSION is deprecated and will be removed in Pillow 9 (2022-01-02). Use __version__ instead.
  from PIL import Image, ImageOps, ImageEnhance, PILLOW_VERSION


<font size = 3>

### gen_supernet

#### - 调用测试

In [2]:
model, sta_num, resolution = gen_supernet(
    flops_minimum = 0, 
    flops_maximum = 300, 
    num_classes = 1000, 
    drop_rate = 0., 
    global_pool = "avg", 
    resunit = False, 
    dil_conv = False, 
    slice = 4, 
    verbose = False, 
    logger = None)

In [ ]:
print(sta_num)
print(resolution)

#### - 网络结构检查

In [ ]:
for name, layer in model.named_sublayers():
    print(name, layer)

<font size = 3>

### gen_childnet

#### - 调用测试

In [ ]:
test_childnet = gen_childnet(
    arch_list = [
        [0], 
        [3], 
        [3, 3], 
        [3, 3], 
        [3, 3, 3], 
        [3, 3], 
        [0]], 
    # arch_list = [
    #     [0], 
    #     [3], 
    #     [3, 3], 
    #     [3, 3], 
    #     [3], 
    #     [3], 
    #     [0]], 
    arch_def = [
        ["ds_r1_k3_s1_e1_c16_se0.25"], 
        ["ir_r1_k3_s2_e4_c24_se0.25"], 
        ["ir_r1_k5_s2_e4_c40_se0.25"] * 2, 
        ["ir_r1_k3_s2_e6_c80_se0.25"] * 2, 
        ["ir_r1_k3_s1_e6_c96_se0.25"] * 3, 
        ["ir_r1_k5_s2_e6_c192_se0.25"] * 2, 
        ["cn_r1_k1_s1_c320_se0.25"]], 
    num_classes = 1000, 
    drop_rate = .2, 
    global_pool = "avg", )

#### - 网络结构检查

In [ ]:
import paddle
test_childnet_model = paddle.Model(test_childnet)
test_childnet_model.summary((16, 3, 224, 224))

#### - 网络运行测试

In [ ]:
import paddle
test_childnet.eval()
re = test_childnet(paddle.randn((16, 3, 224, 224)))
print([vec.index(max(vec)) for vec in re.tolist()])

## ***其他***

#### - import测试

In [3]:
from Paddle_Cream.lib.models.MetaMatchingNetwork import MetaMatchingNetwork

<font size = 3>

### MetaMatchingNetwork

#### - 实例化测试

In [4]:
class no:
    def __init__(self, include = False) -> None:
        if include:
            self.SUPERNET = no()
            self.SUPERNET.PICK_METHOD = "meta"
cfg = no(True)

In [5]:
MetaMN = MetaMatchingNetwork(cfg)

进行以下测试前请先在[lib/models/structures](Paddle_Cream\lib\models\structures)的测试中生成超网络

#### - forward_training调用测试

In [6]:
import paddle
x = paddle.randn((16, 3, 224, 224))
stu_cand = [[0], [3], [3, 3], [3, 3], [3, 3, 3], [3, 3], [0]]
tch_cand = [[0], [2], [4, 2], [3, 0], [0, 3, 1], [3, 5], [0]]
super_model = model
meta = .5

In [7]:
loss = MetaMN.forward_training(
    x = x, 
    model = super_model, 
    random_cand = stu_cand, 
    teacher_cand = tch_cand, 
    meta_value = meta)

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\nn\layer\norm.py:640: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


In [ ]:
print(loss)

#### - calculate_1st_gradient调用测试

In [8]:
from paddle.optimizer import SGD
optim = SGD(learning_rate = 10., parameters = super_model.parameters())

In [9]:
grad = MetaMN.calculate_1st_gradient(
    kd_loss = loss, 
    model = super_model, 
    random_cand = stu_cand, 
    optimizer = optim)

C:\Users\LibrarristShalinward\AppData\Local\Programs\Python\Python38\lib\site-packages\paddle\fluid\dygraph\varbase_patch_methods.py:382: UserWarning: 
tensor.grad will return the tensor value of the gradient. This is an incompatible upgrade for tensor.grad API.  It's return type changes from numpy.ndarray in version 2.0 to paddle.Tensor in version 2.1.0.  If you want to get the numpy value of the gradient, you can use :code:`x.grad.numpy()` 
  warnings.warn(warning_msg)


In [ ]:
for g in grad:
    if type(g) == type(None):
        print(g)
    else:
        print(g.norm())

#### - simulate_sgd_update调用测试

In [ ]:
students_weight = [
    MetaMN.simulate_sgd_update(
        p, grad_item, optim) for p, grad_item in zip(
        model.rand_parameters(stu_cand), grad)]

In [ ]:
for weight_del in students_weight:
    print(weight_del)

#### - update_student_weights_only

In [10]:
for i, param in enumerate(super_model.rand_parameters(stu_cand)):
    if i == 23:
        print(param[0][0])
        break

Tensor(shape=[1, 1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-0.08248105]])


In [11]:
for i, param in enumerate(super_model.rand_parameters(tch_cand)):
    if i == 23:
        print(param[0][0])
        break

Tensor(shape=[1, 1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[0.24557313]])


In [12]:
MetaMN.update_student_weights_only(stu_cand, optim, super_model)

In [13]:
for i, param in enumerate(super_model.rand_parameters(stu_cand)):
    if i == 23:
        print(param[0][0])
        break

Tensor(shape=[1, 1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[-0.08249819]])


In [14]:
for i, param in enumerate(super_model.rand_parameters(tch_cand)):
    if i == 23:
        print(param[0][0])
        break

Tensor(shape=[1, 1], dtype=float32, place=CPUPlace, stop_gradient=False,
       [[0.24557313]])


# [lib/utils](Paddle_Cream\lib\utils)

## ***[builder_util.py](Paddle_Cream\lib\utils\builder_util.py)***

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.builder_util import decode_block_str, init_weight_goog, efficientnet_init_weights

<font size = 3>

###  decode_block_str

In [ ]:
str_eg = "ir_r1_k3_s1_e4_c80_se0.25"
block_args, num_repeat = decode_block_str(str_eg)

In [ ]:
for item in block_args.items():
    print(item)
print(num_repeat)

<font size = 3>

### init_weight_goog

#### - 调用测试：CondConv2D

In [ ]:
from Paddle_Cream.lib.utils.pimm.models import CondConv2D
test_cond = CondConv2D(3, 8, 5)

In [ ]:
print(test_cond.weight[0][:10])
print(test_cond.bias[0])

In [ ]:
init_weight_goog(test_cond)

In [ ]:
print(test_cond.weight[0][:10])
print(test_cond.bias[0])

#### - 调用测试：Conv2D

In [ ]:
from paddle.nn import Conv2D
test_conv = Conv2D(16, 24, 3, 2, 1)

In [ ]:
print(test_conv.weight[0][:10])
print(test_conv.bias)

In [ ]:
init_weight_goog(test_conv)

In [ ]:
print(test_conv.weight[0][:10])
print(test_conv.bias)

#### - 调用测试：BatchNorm2D

In [ ]:
from paddle.nn import BatchNorm2D
from paddle.framework import create_parameter, ParamAttr
from paddle.nn.initializer import Normal
test_batch = BatchNorm2D(32)
test_batch.weight = create_parameter(
    test_batch.weight.shape, 
    test_batch.weight.dtype, 
    test_batch.weight.name, 
    ParamAttr(initializer = Normal()))
test_batch.bias = create_parameter(
    test_batch.bias.shape, 
    test_batch.bias.dtype, 
    test_batch.bias.name, 
    ParamAttr(initializer = Normal()))

In [ ]:
print(test_batch.weight)
print(test_batch.bias)

In [ ]:
init_weight_goog(test_batch, last_bn = [])

In [ ]:
print(test_batch.weight)
print(test_batch.bias)

#### - 调用测试：Linear

In [ ]:
from paddle.nn import Linear
test_linear = Linear(8, 8)

In [ ]:
print(test_linear.weight[0])
print(test_linear.bias)

In [ ]:
init_weight_goog(test_linear)

In [ ]:
print(test_linear.weight[0])
print(test_linear.bias)

<font size = 3>



<font size = 3>

### efficientnet_init_weights

In [ ]:
efficientnet_init_weights(test_stages[0])

## ***[flops_table.py](Paddle_Cream\lib\utils\flops_table.py)***

#### - import测试

In [ ]:
from Paddle_Cream.lib.utils.flops_table import FlopsEst

#### - 实例化测试

测试前请先在[lib/models/structures](Paddle_Cream\lib\models\structures)的测试中生成超网络

In [ ]:
model_est = FlopsEst(model, input_shape = (2, 3, 160, 160))

In [ ]:
model_est.flops_dict

In [ ]:
model_est.params_dict

#### - 调用测试

In [ ]:
model_est.get_flops([
        [0], 
        [3], 
        [3, 3], 
        [3, 3], 
        [3, 3, 3], 
        [3, 3], 
        [0]])

In [ ]:
model_est.get_params([
        [0], 
        [3], 
        [3, 3], 
        [3, 3], 
        [3, 3, 3], 
        [3, 3], 
        [0]])

# 